In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
#from keras.layers.recurrent import LSTM,GRU
#from keras.layers.core import Dense, Activation, Dropout
from sklearn.metrics import roc_auc_score, accuracy_score
#from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model
from nltk import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

Using TensorFlow backend.


In [2]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv?dl=0

--2020-03-22 04:14:49--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv [following]
--2020-03-22 04:14:49--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3dafae15344cbd4f5746303d62.dl.dropboxusercontent.com/cd/0/inline/A0UokCDAvOUG6DeVE8uLDmvW3W62HZCqmZqTsgdcvXT-EqElLwTV-ogUierea8Inf3E7KrL0c8qqgRfWLHiaciwyekNtDHos07X-wQoFJ2BGgQ/file# [following]
--2020-03-22 04:14:49--  https://uc3dafae15344cbd4f5746303d62.dl.dropboxusercontent.com/cd/0/inline/A0UokCDAvOUG6DeVE8uLDmvW3W62HZCqmZqTsgdcvXT-EqElLwTV-ogUie

In [3]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2020-03-22 04:14:55--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2020-03-22 04:14:55--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2020-03-22 04:14:55--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [4]:
!unzip /content/glove.42B.300d.zip

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [0]:
embedding_index ={}

f = open('/content/glove.42B.300d.txt', encoding='utf-8')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype ='float32')
  embedding_index[word] = coefs
f.close()

In [0]:
Data = pd.read_csv(r'/content/train.csv?dl=0')

In [7]:
print(Data.shape)
Data.head()

(1306122, 3)


,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [8]:
X = Data['question_text']
Y = Data['target']
x_train, x_val, y_train, y_val = train_test_split(X,Y,test_size=0.2)
x_train.shape, y_train.shape, x_val.shape, y_val.shape

((1044897,), (1044897,), (261225,), (261225,))

In [9]:
class_weight = class_weight.compute_class_weight('balanced', 
                                                 np.unique(y_train), 
                                                    y_train)
print(class_weight)
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

[0.53291278 8.09583469]


{0: 0.5329127752549053, 1: 8.095834689228766}

In [10]:
x_train

641132     Why does the western media ignore human rights...
274307     When does the Federal Reserve/Reserve Bank dec...
1043038    Is it a red flag if my boyfriend told me he do...
929035                How do I disable Briefing for Samsung?
930724     Is the Queen of England more important/has mor...
                                 ...                        
858689     Can you name any words that start with the let...
1065790    Which is the best company, if any, that hire o...
353154     Can I study in TU9 (Germany) with a grade of 6...
412651     What are some of the funniest things a toddler...
774942     When will the Disclosure Project become a real...
Name: question_text, Length: 1044897, dtype: object

In [11]:
import nltk
nltk.download('punkt') 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

412


In [13]:
print(np.percentile(sent_lens,99))
np.percentile(sent_lens,95)

45.0


31.0

In [14]:
max_len=45

tk=Tokenizer(char_level=False,split=' ')
tk.fit_on_texts(x_train)
print(x_train.shape)

(1044897,)


In [15]:
seq_train=tk.texts_to_sequences(x_train)
seq_val=tk.texts_to_sequences(x_val)
print(len(seq_val), len(seq_train))
seq_train[0]

261225 1044897


[16, 26, 1, 786, 328, 2125, 235, 816, 8391, 6, 57]

In [16]:
vocab_size=len(tk.word_index)
vocab_size

196231

In [17]:
seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_val_matrix=sequence.pad_sequences(seq_val,maxlen=max_len)
seq_train_matrix.shape, seq_val_matrix.shape

((1044897, 45), (261225, 45))

In [0]:
embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embedding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

In [19]:
inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(layer1)
layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(layer2)
layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(layer3)
dense1 = Dense(25,activation='relu')(layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)










Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [20]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 45)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 45, 300)           58869600  
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               1665024   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3289

In [0]:
filepath='/content/weights.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True)

In [0]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [23]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [24]:
model.fit(seq_train_matrix,y_train,
              epochs=10,
              class_weight={0: 0.5329682268617981, 1: 8.083058714318868},
              batch_size=10000,
              validation_data=(seq_val_matrix,y_val),
              callbacks = [earlystop, checkpoint])



Train on 1044897 samples, validate on 261225 samples
Epoch 1/10
1044897/1044897 [==============================] - 4545s 4ms/step - loss: 0.3424 - acc: 0.8451 - val_loss: 0.3093 - val_acc: 0.8606

Epoch 00001: val_loss improved from inf to 0.30928, saving model to /content/weights.h5
Epoch 2/10
1044897/1044897 [==============================] - 4514s 4ms/step - loss: 0.2818 - acc: 0.8789 - val_loss: 0.3165 - val_acc: 0.8611

Epoch 00002: val_loss did not improve from 0.30928
Epoch 3/10
1044897/1044897 [==============================] - 4509s 4ms/step - loss: 0.2624 - acc: 0.8873 - val_loss: 0.2406 - val_acc: 0.8961

Epoch 00003: val_loss improved from 0.30928 to 0.24062, saving model to /content/weights.h5
Epoch 4/10
1044897/1044897 [==============================] - 4519s 4ms/step - loss: 0.2448 - acc: 0.8929 - val_loss: 0.2143 - val_acc: 0.9113

Epoch 00004: val_loss improved from 0.24062 to 0.21429, saving model to /content/weights.h5
Epoch 5/10
1044897/1044897 [==================

In [0]:
best_model = load_model('/content/weights.h5')

In [26]:
best_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 45)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 45, 300)           58869600  
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               1665024   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3289

In [27]:
p = best_model.predict(seq_val_matrix)
roc_auc_score(y_val,p)

0.9609479467549491